# Complete exploratory data analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import re

In [ ]:
matplotlib.rcParams['figure.figsize'] = (10, 5)
matplotlib.rcParams['font.size'] = 12

## Load the data

In [ ]:
market_train_df = pd.read_feather('../input/market_train_df.feather')

In [ ]:
news_train_df   = pd.read_feather('../input/news_train_df.feather')

In [ ]:
print('Market train shape: ',market_train_df.shape)
print('News train shape: ', news_train_df.shape)

In [ ]:
market_train_df.head(5)

In [ ]:
market_train_df.describe()

In [ ]:
news_train_df.head(5)

In [ ]:
news_train_df.describe()

## Explore news data

### Evolutions over time

In [ ]:
# Sort values by time then extract date
news_train_df = news_train_df.sort_values(by='time')
news_train_df['date'] = news_train_df['time'].dt.date

In [ ]:
# Function to plot time series data
def plot_vs_time(data_frame, column, calculation='mean', span=10):
    if calculation == 'mean':
        group_temp = data_frame.groupby('date')[column].mean().reset_index()
    if calculation == 'count':
        group_temp = data_frame.groupby('date')[column].count().reset_index()
    if calculation == 'nunique':
        group_temp = data_frame.groupby('date')[column].nunique().reset_index()
    group_temp = group_temp.ewm(span=span).mean()
    fig = plt.figure(figsize=(10,3))
    plt.plot(group_temp['date'], group_temp[column])
    plt.xlabel('Time')
    plt.ylabel(column)
    plt.title('%s versus time' %column)

In [ ]:
plot_vs_time(news_train_df, 'sourceId', calculation='count', span=10)
plt.title('News count vs time')
plt.ylabel('Count')

There is a maximum peak every quarter (time for quaterly financial report) and a minimum peak at the end of the year (time for Christmast holliday.)

In [ ]:
# Plot time evolution of several parameters

columns = ['urgency', 'takeSequence', 'companyCount','marketCommentary','sentenceCount',\
           'firstMentionSentence','relevance','sentimentClass','sentimentWordCount','noveltyCount24H', 'volumeCounts24H']

for column in columns:
    plot_vs_time(news_train_df, column)

### Time delay

In [ ]:
time_delay = (pd.to_datetime(news_train_df['time']) - pd.to_datetime(news_train_df['firstCreated']))
time_delay_log10 = np.log10(time_delay.dt.total_seconds()/60+1)

In [ ]:
plt.hist(time_delay_log10, bins=np.arange(0,2.5,0.25), rwidth=0.7)
plt.xlabel('$Log_{10}$(Time delay in minutes +1)')
plt.ylabel('Counts')
plt.title('Delay time distribution')

In [ ]:
time_delay_min = time_delay.dt.total_seconds()/60
time_delay_df = time_delay_min.to_frame().join(news_train_df['date'].to_frame())
time_delay_df.columns = ['delay','date']
plot_vs_time(time_delay_df, 'delay')
plt.ylabel('Delay (minutes)')

### Urgency

In [ ]:
urgency_count = news_train_df.groupby('urgency')['sourceId'].count()
urgency_count = urgency_count/urgency_count.sum()
print('Urgency ratio')
urgency_count.sort_values(ascending=True)

### Take sequence

In [ ]:
take_sequence = news_train_df.groupby('takeSequence')['sourceId'].count()

In [ ]:
take_sequence = take_sequence.sort_values(ascending= False)
take_sequence[:10].plot.barh()
plt.xlabel('Count')
plt.ylabel('Take sequence')
plt.title('Top 10 take sequence')
plt.gca().invert_yaxis()

### Providers

In [ ]:
provider_count = news_train_df.groupby('provider')['sourceId'].count()

In [ ]:
provider_sort = provider_count.sort_values(ascending= False)
provider_sort[:10].plot.barh()
plt.xlabel('Count')
plt.ylabel('Provider')
plt.title('Top 10 news provider')
plt.gca().invert_yaxis()

### Subjects

In [ ]:
# Extract data from a single cell
def contents_to_list(contents):
    text = contents[1:-1]
    text = re.sub(r",",' ',text)
    text = re.sub(r"'","", text)
    text_list = text.split('  ')
    return text_list

# Put data from columns into dict
def get_content_dict(content_column):
    content_dict = {}
    for k in content_column.keys():
        this_cell = content_column[k]
        content_list = contents_to_list(this_cell)        
        for content in content_list:
            if content in content_dict.keys():
                content_dict[content] += 1
            else:
                content_dict[content] = 1
    return content_dict


In [ ]:
subjects = news_train_df.sample(n=10000, random_state=1)['subjects']
subjects_dict = get_content_dict(subjects)

In [ ]:
subjects_df = pd.Series(subjects_dict).sort_values(ascending=False)
subjects_df[:15].plot.barh()
plt.ylabel('Subjects')
plt.xlabel('Counts')
plt.title('Top subjects for 10k data')
plt.gca().invert_yaxis()

### Audiences

In [ ]:
audiences = news_train_df.sample(n=10000, random_state=1)['audiences']
audiences_dict = get_content_dict(audiences)

In [ ]:
audiences_df = pd.Series(audiences_dict).sort_values(ascending=False)
audiences_df[:15].plot.barh()
plt.ylabel('Audiences')
plt.xlabel('Counts')
plt.title('Top audiences for 10k data')
plt.gca().invert_yaxis()

### Company Count

In [ ]:
news_train_df['companyCount'].hist(bins=np.arange(0,30,1))
plt.xlabel('Company count')
plt.title('Company count distribution')

### Head line tag

In [ ]:
head_line = news_train_df.groupby('headlineTag')['sourceId'].count()

In [ ]:
head_line_sort = head_line.sort_values(ascending= False)
head_line_sort[:10].plot.barh()
plt.xlabel('Count')
plt.ylabel('Head line')
plt.title('Top 10 head lines')
plt.gca().invert_yaxis()

Most headlines are blank. This properties may not be important.

### First sentence - Urgency - relevance - sentiment Word Count

**First sentence and urgency**

In [ ]:
news_train_df['firstMentionSentence'].hist(bins=np.arange(0,20,1))
plt.xlabel('First mention sentence')
plt.ylabel('Count')
plt.title('First mention sentence distribution')

In [ ]:
sentence_urgency = news_train_df.groupby('firstMentionSentence')['urgency'].mean()
sentence_urgency.head(5)

**First sentence and relevance**

In [ ]:
news_train_df['relevance'].hist(bins=np.arange(0,1.01,0.05))
plt.xlabel('Relevance')
plt.ylabel('Count')
plt.title('Relevance distribution')

In [ ]:
sentence_relevance = news_train_df.groupby('firstMentionSentence')['relevance'].mean()
sentence_relevance[:15].plot.barh()
plt.xlabel('Relevance')
plt.title('Relevance by sentence')
plt.gca().invert_yaxis()

**Sentiment word count and relevance**

In [ ]:
sentimentWordCount = news_train_df.groupby('sentimentWordCount')['sourceId'].count().reset_index()
plt.plot(sentimentWordCount['sentimentWordCount'], sentimentWordCount['sourceId'])
plt.xlim(0,300)
plt.xlabel('Sentiment words count')
plt.ylabel('Count')
plt.title('Sentiment words count distribution')

In [ ]:
sentimentWordRatio = news_train_df.groupby('sentimentWordCount')['relevance'].mean()
plt.plot(sentimentWordRatio)
plt.xlim(0,2000)
plt.ylabel('Relevance')
plt.xlabel('Sentiment word count')
plt.title('Sentiment word count and relevance')

**Sentiment ratio**

In [ ]:
news_train_df['sentimentRatio'] = np.float32(news_train_df['sentimentWordCount']/news_train_df['wordCount'])
news_train_df['sentimentRatio'].hist(bins=np.linspace(0,1.001,40))
plt.xlabel('Sentiment ratio')
plt.ylabel('Count')
plt.title('Sentiment ratio distribution')

In [ ]:
news_train_df.sample(n=10000, random_state=1).plot.scatter('sentimentRatio', 'relevance')
plt.title('Relevance vs sentiment ratio of 10k samples')

### Asset name

In [ ]:
asset_name = news_train_df.groupby('assetName')['sourceId'].count()
print('Total number of assets: ',news_train_df['assetName'].nunique())

In [ ]:
asset_name = asset_name.sort_values(ascending=False)
asset_name[:10].plot.barh()
plt.gca().invert_yaxis()
plt.xlabel('Count')
plt.title('Top 10 assets news')

In [ ]:
for i, j in zip([-1, 0, 1], ['negative', 'neutral', 'positive']):
    df_sentiment = news_train_df.loc[news_train_df['sentimentClass'] == i, 'assetName']
    print(f'Top mentioned companies for {j} sentiment are:')
    print(df_sentiment.value_counts().head(5))
    print('')

### Remove outliers and plot correlation

In [ ]:
# Function to remove outliers
def remove_outliers(data_frame, column_list, low=0.02, high=0.98):
    temp_frame = data_frame
    for column in column_list:
        this_column = data_frame[column]
        quant_df = this_column.quantile([low,high])
        low_limit = quant_df[low]
        high_limit = quant_df[high]
        temp_frame[column] = data_frame[column].clip(lower=low_limit, upper=high_limit)
    return temp_frame

In [ ]:
# Remove outlier
columns_outlier = ['takeSequence', 'bodySize', 'sentenceCount', 'wordCount', 'sentimentWordCount', 'firstMentionSentence','noveltyCount12H',\
                  'noveltyCount24H', 'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D', 'volumeCounts12H', 'volumeCounts24H',\
                  'volumeCounts3D','volumeCounts5D','volumeCounts7D']
news_rmv_outlier = remove_outliers(news_train_df, columns_outlier)

In [ ]:
# Plot correlation
columns_corr = ['urgency', 'takeSequence', 'companyCount','marketCommentary','sentenceCount',\
           'firstMentionSentence','relevance','sentimentClass','sentimentWordCount','noveltyCount24H',\
           'noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D','volumeCounts24H','volumeCounts3D','volumeCounts5D','volumeCounts7D']
colormap = plt.cm.RdBu
plt.figure(figsize=(18,15))
sns.heatmap(news_rmv_outlier[columns_corr].astype(np.float16).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('Pair-wise correlation')

## Explore market data

In [ ]:
print('Check null data:')
market_train_df.isna().sum()

**Some preprocessing:**
* Sort data in chronological order
* All NAN data comes from the market adjusted column. We fill them up with the raw value data

In [ ]:
# Sort data
market_train_df = market_train_df.sort_values('time')
market_train_df['date'] = market_train_df['time'].dt.date

# Fill nan
market_train_fill = market_train_df
column_market = ['returnsClosePrevMktres1','returnsOpenPrevMktres1','returnsClosePrevMktres10', 'returnsOpenPrevMktres10']
column_raw = ['returnsClosePrevRaw1', 'returnsOpenPrevRaw1','returnsClosePrevRaw10', 'returnsOpenPrevRaw10']
for i in range(len(column_raw)):
    market_train_fill[column_market[i]] = market_train_fill[column_market[i]].fillna(market_train_fill[column_raw[i]])

### Plot data versus time

In [ ]:
plot_vs_time(market_train_fill, 'assetCode', 'count')
plt.title('Number of asset codes versus time')

In [ ]:
# Inspired by https://www.kaggle.com/artgor/eda-feature-engineering-and-everything
for i in [0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95]:
    price_df = market_train_fill.groupby('date')['close'].quantile(i).reset_index()
    plt.plot(price_df['date'], price_df['close'], label='%.2f quantile' %i)
plt.legend(loc='best')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title('Market close price by quantile')

In [ ]:
for i in [0.05, 0.25, 0.5, 0.75, 0.95]:
    price_df = market_train_fill.groupby('date')['returnsClosePrevRaw1'].quantile(i).reset_index()
    plt.plot(price_df['date'], price_df['returnsClosePrevRaw1'], label='%.2f quantile' %i)
plt.legend(loc='best')
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('returnsClosePrevRaw1 by quantile')


In [ ]:
for i in [0.05, 0.25, 0.5, 0.75, 0.95]:
    price_df = market_train_fill.groupby('date')['returnsOpenPrevRaw10'].quantile(i).reset_index()
    plt.plot(price_df['date'], price_df['returnsOpenPrevRaw10'], label='%.2f quantile' %i)
plt.legend(loc=1)
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('returnsOpenPrevRaw10 by quantiles')

In [ ]:
for i in [0.05, 0.25, 0.5, 0.75, 0.95]:
    price_df = market_train_fill.groupby('date')['returnsOpenPrevMktres10'].quantile(i).reset_index()
    plt.plot(price_df['date'], price_df['returnsOpenPrevMktres10'], label='%.2f quantile' %i)
plt.legend(loc=1)
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('returnsOpenPrevMktres10 by quantiles')

In [ ]:
for i in [0.05, 0.25, 0.5, 0.75, 0.95]:
    price_df = market_train_fill.groupby('date')['returnsOpenNextMktres10'].quantile(i).reset_index()
    plt.plot(price_df['date'], price_df['returnsOpenNextMktres10'], label='%.2f quantile' %i)
plt.legend(loc=1)
plt.xlabel('Time')
plt.ylabel('Value')
plt.title('returnsOpenNextMktres10 by quantiles')

In [ ]:
for i in [0.05, 0.25, 0.5, 0.75, 0.95]:
    price_df = market_train_fill.groupby('date')['volume'].quantile(i).reset_index()
    plt.plot(price_df['date'], price_df['volume'], label='%.2f quantile' %i)
plt.legend(loc='best')
plt.xlabel('Time')
plt.ylabel('Volumes')
plt.title('Market trade volumes by quantile')

### Difference between raw values and market adjusted values

Let see if there's any difference between raw return and market adjusted return

In [ ]:
column_mkt_raw_diff = []
for i in range(len(column_market)):
    this_raw = column_raw[i]
    this_market = column_market[i]
    new_column_name = 'mkt_raw_diff'+this_raw.replace('returns','').replace('Raw','')
    column_mkt_raw_diff.append(new_column_name)
    market_train_fill[new_column_name] = market_train_fill[this_market] - market_train_fill[this_raw]

In [ ]:
market_train_fill[column_mkt_raw_diff].describe()

The difference between raw return and market adjusted returns are negligible, but there are some extreme values. Those values are noise and needs to be taken care of

### Asset codes

In [ ]:
assetCode_df = market_train_df.groupby('assetCode')['volume'].sum().sort_values(ascending=False)
print('There are %i unique asset code' %len(assetCode_df))

In [ ]:
unknown_name = market_train_fill[market_train_fill['assetName']=='Unknown']
unknown_count = unknown_name['assetCode'].value_counts().sort_values(ascending=False)

In [ ]:
print('There are %i unique asset code with unknown asset name' %len(unknown_count))

In [ ]:
unknown_count[:15].plot.barh()
plt.ylabel('assetCode')
plt.xlabel('Counts')
plt.title('Top 15 asset code with Unknown asset name')
plt.gca().invert_yaxis()

In [ ]:
assetCode_df[:15].plot.barh()
plt.ylabel('assetCode')
plt.xlabel('Trading volume')
plt.title('Top 15 asset code by volume')
plt.gca().invert_yaxis()

### Asset Name

In [ ]:
assetName_Volume = market_train_df.groupby('assetName')['volume'].sum().sort_values(ascending=False)
assetName_Volume[:15].plot.barh()
plt.ylabel('assetName')
plt.xlabel('Trading volume')
plt.title('Top 15 asset name by volume')
plt.gca().invert_yaxis()

The volume ranking by coorperation seems to be the same as the rank of asset codes they own, e.g. the one with most popular codes has the most trading volume

In [ ]:
assetName_code = market_train_df.groupby('assetName')['assetCode'].nunique().reset_index().sort_values(by='assetCode',ascending=False)

In [ ]:
assetCodeCount = assetName_code.groupby('assetCode')['assetName'].count().reset_index()
assetCodeCount.columns = ['assetCodeNo', 'counts']
assetCodeCount.head()

**The vast majority of companies has only one asset code**. One '*company*' that has 110 actually is the  'Unknown' category. Magically, some companies don't even have any asset code. Currently I have no explanation for this.

### Correlations

In [ ]:
columns_corr_market = ['volume', 'open', 'close','returnsClosePrevRaw1','returnsOpenPrevRaw1',\
           'returnsClosePrevMktres1','returnsOpenPrevMktres1','returnsClosePrevMktres10','returnsOpenPrevRaw10',\
           'returnsClosePrevMktres10', 'returnsOpenPrevMktres10', 'returnsOpenNextMktres10']
colormap = plt.cm.RdBu
plt.figure(figsize=(18,15))
sns.heatmap(market_train_fill[columns_corr_market].astype(np.float16).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('Pair-wise correlation')

### Dig deeper to a single asset

Let's take a closer look to a single asset. Here I choose the one with largest trading volumen: 'Bank of America Corp'

In [ ]:
assetCode = 'Bank of America Corp'
thisAssetMark_df = market_train_fill[market_train_fill['assetName']==assetCode].sort_values(by='date',ascending=True) 
thisAssetMark_df['diff_open_close'] = thisAssetMark_df['open'] - thisAssetMark_df['close']
thisAssetNews_df = news_rmv_outlier[news_rmv_outlier['assetName']==assetCode]
# Trading volume vs time
thisAssetMark_df.plot(x='date', y='volume')
plt.title('Trading volume vs time')
# Price vs time
thisAssetMark_df.plot(x='date', y='open')
plt.title('Open price vs time')
# Return vs time
thisAssetMark_df.plot(x='date', y=['returnsOpenPrevRaw1', 'returnsOpenPrevRaw10','returnsOpenNextMktres10'], alpha=0.8)
plt.title('Return vs time')


It can be seen that trading volume is strongly associated with price, i.e. trade increase when price hits bottom. Return is also strongly fluctuated at such time

In [ ]:
news_volume = thisAssetNews_df.groupby('date')['sourceId'].count().reset_index()
news_volume = news_volume.ewm(span=10).mean()
news_volume.plot(x='date',y='sourceId')
plt.title('News volume vs time')

In [ ]:
news_urgency = thisAssetNews_df.groupby('date')['urgency'].mean().reset_index()
news_urgency = news_urgency.ewm(span=10).mean()
news_urgency.plot(x='date',y='urgency')
plt.title('News urgency vs time')

The news increases in volumes and urgency as price drops

In [ ]:
news_relevance = thisAssetNews_df.groupby('date')['relevance'].mean().reset_index()
news_relevance = news_relevance.ewm(span=10).mean()
news_relevance.plot(x='date',y='relevance')
plt.title('Relevance vs time')

In [ ]:
news_sentiment = thisAssetNews_df.groupby('date')['sentimentClass','sentimentNegative','sentimentNeutral','sentimentPositive'].mean().reset_index()
news_sentiment = news_sentiment.ewm(span=10).mean()
news_sentiment.plot(x='date',y=['sentimentClass','sentimentNegative','sentimentNeutral','sentimentPositive'], alpha=0.8)
plt.title('Sentiment vs time')

Sentiments are mostly negative. Sentiment drops as price drops, which is expected.

Now let's merge the news and market data and see their correlations

In [ ]:
# Merge news and market data. Only keep numeric columns
thisAssetMark_number = thisAssetMark_df[columns_corr_market+['date']]
thisAssetMark_number = thisAssetMark_number.groupby('date').mean().reset_index()
thisAssetNews_number = thisAssetNews_df[columns_corr+['date']]
thisAssetNews_number = thisAssetNews_number.groupby('date').mean().reset_index()
thisAssetNews_number['news_volume'] = thisAssetNews_df.groupby('date')['sourceId'].count().reset_index()['sourceId']
thisAssetMerge = pd.merge(thisAssetMark_number, thisAssetNews_number, how='left', on = 'date')

In [ ]:
columns_corr_merge = ['volume','open','close','returnsOpenPrevRaw1','returnsOpenPrevMktres1','returnsOpenPrevRaw10','returnsOpenPrevMktres10',\
                     'returnsOpenNextMktres10','news_volume','urgency','sentenceCount','relevance','sentimentClass',\
                     'noveltyCount24H','noveltyCount5D','volumeCounts24H','volumeCounts5D']
colormap = plt.cm.RdBu
plt.figure(figsize=(18,15))
sns.heatmap(thisAssetMerge[columns_corr_merge].astype(np.float16).corr(), linewidths=0.1, vmax=1.0, vmin=-1., square=True, cmap=colormap, linecolor='white', annot=True)
plt.title('Pair-wise correlation market and news')

References: Nguyen Dang Minh, PhD